This is the exact notebook I used for the submission with 0.85 score on the leaderboard, and uses the nadaraya watson method for predicting properties. File main.py in this directory is the file where I wrote my code first, and then copy pasted on kaggle in the form of a notebook which is this.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM/config.json
/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM/merges.txt
/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM/tokenizer.json
/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM/vocab.json
/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM/tokenizer_config.json
/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM/model.safetensors
/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM/special_tokens_map.json
/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM/added_tokens.json
/kaggle/input/neurips-open-polymer-prediction-2025/sample_submission.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train.csv
/kaggle/input/neurips-open-polymer-prediction-2025/test.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset2.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv
/kaggle/input/neurips-open-polymer-prediction-2025/train_

In [2]:
from sentence_transformers import SentenceTransformer
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import numpy as np

2025-08-14 02:22:31.197467: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755138151.370965      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755138151.421732      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# model = SentenceTransformer("Derify/ChemMRL-alpha")

In [4]:
# model_path = 'kaggle/input/models-derify-chemmrl-alpha/pytorch/default/1/models--Derify--ChemMRL-alpha/snapshots/10bace4387f5ce86181c382b110ece9cd55d9dcc'
# model_path = '../input/models-derify-chemmrl-alpha/pytorch/default/1/models--Derify--ChemMRL-alpha/snapshots/10bace4387f5ce86181c382b110ece9cd55d9dcc'
# model_path = '/kaggle/working/.cache/huggingface/hub/models--Derify--ChemMRL-alpha/snapshots/10bace4387f5ce86181c382b110ece9cd55d9dcc'
model_path = '/kaggle/input/c/transformers/default/1/ChemBERTa-77M-MLM'
model = SentenceTransformer(model_path)

In [5]:
# read data
df_train = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train.csv')
df_test = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/test.csv')
print(f'Train shape: {df_train.shape}')
print(f'Test shape: {df_test.shape}')

Train shape: (7973, 7)
Test shape: (3, 2)


### Simple model, Nadaraya-Watson regression for each property

In [6]:
# get all the subsets of the df, where each subset has all the information for one property
data_tg = df_train.loc[~df_train['Tg'].isna(), ['id', 'SMILES', 'Tg']]
data_ffv = df_train.loc[~df_train['FFV'].isna(), ['id', 'SMILES', 'FFV']]
data_tc = df_train.loc[~df_train['Tc'].isna(), ['id', 'SMILES', 'Tc']]
data_density = df_train.loc[~df_train['Density'].isna(), ['id', 'SMILES', 'Density']]
data_rg = df_train.loc[~df_train['Rg'].isna(), ['id', 'SMILES', 'Rg']]

In [7]:
embeddings = model.encode(list(df_train['SMILES']))

Batches:   0%|          | 0/250 [00:00<?, ?it/s]

In [8]:
# class holding all information of the regressor we are creating in one place
class NWRegressor():
    def __init__(self, df_train, model, kernel_bandwidth):
        # store the model as an attribute
        self.model = model
        # store as an attribute, the bandwidth of the kernel
        self.bandwidth = kernel_bandwidth
        # the df_train is also an integral part of the object, so should be stored as an attribute
        self.df_train = df_train
        # store all the data-subset attributes
        self.data_tg = df_train.loc[~df_train['Tg'].isna(), ['id', 'SMILES', 'Tg']]
        self.data_ffv = df_train.loc[~df_train['FFV'].isna(), ['id', 'SMILES', 'FFV']]
        self.data_tc = df_train.loc[~df_train['Tc'].isna(), ['id', 'SMILES', 'Tc']]
        self.data_density = df_train.loc[~df_train['Density'].isna(), ['id', 'SMILES', 'Density']]
        self.data_rg = df_train.loc[~df_train['Rg'].isna(), ['id', 'SMILES', 'Rg']]

    # separate method to initialize and assign embeddings because sometimes we may have them already computed and just want to assign so there is no need to do this in __init__
    def store_embeddings(self):
        # store the embeddings of all the smiles in df_train
        self.embeddings = self.model.encode(self.df_train['SMILES']) # shape: (nrow(df_train), 1024) = (7973, 1024)

    def kernel_fn(self, x):
        numerator = -(x * x)
        denominator = 2 * self.bandwidth * self.bandwidth
        return np.exp(numerator/denominator)
    
    def distance(self, arr1, arr2):
        # Compute pairwise distances (Euclidean by default)
        arr1_exp = arr1[:, np.newaxis, :]  # (x, 1, 120)
        arr2_exp = arr2[np.newaxis, :, :]  # (1, y, 120)
        sq_diff = (arr1_exp - arr2_exp) ** 2  # (x, y, 120)
        sq_distances = np.sum(sq_diff, axis=2)  # (x, y)
        distances = np.sqrt(sq_distances)  # (x, y)
        return distances

    # data_property is the dataframe corresponding to a certain property for which predictions are being made
    def predict_property(self, test_embeddings, data_property):
        # find out the distances between test_embeddings and self.embeddings' subset of data_property
        dists = self.distance(self.embeddings[data_property.index], test_embeddings) # shape: (nrow(data_property), test_embeddings)
        weights = self.kernel_fn(dists) # pass the mere distances to kernel to get weights
        # now, we scale the kernel-passed weights so that for each test_embedding, the weights' sum is 1
        weights_scaled = weights/weights.sum(axis = 0)
        # now we just tranpose as test_embeddings being as rownames are more intuitive
        weights_scaled = weights_scaled.T # shape: (test_embeddings,nrow(data_property))
        # now, just perform dot product with tg values vector to get tg value for each test_embedding
        train_tg_values = data_property.iloc[:, -1].values
        output_property_values = np.matmul(weights_scaled, train_tg_values)
        return output_property_values # shape: (test_embeddings,)

        # HERE... go through the details of the nadaraya-watson implementation. Where scaling, softmax etc. Then, finally
        # use the scaled weights to compute the avg value of tg.

    # predict on a list of smiles
    def predict(self, smiles):
        # create empty numpy arrays on which we can append as we get more and more predictions
        tg = np.empty(0)
        ffv = np.empty(0)
        tc = np.empty(0)
        density = np.empty(0)
        rg = np.empty(0)
        # loop is for being able to treat big list of smiles in parts
        for i in range(0, len(smiles), 1000):
            smiles_subset = smiles[i:i+1000]
            test_embeddings = self.model.encode(smiles_subset)
            # call the predict method for each property and return the values
            tg = np.concatenate([tg, self.predict_property(test_embeddings, self.data_tg)])
            ffv = np.concatenate([ffv, self.predict_property(test_embeddings, self.data_ffv)])
            tc = np.concatenate([tc, self.predict_property(test_embeddings, self.data_tc)])
            density = np.concatenate([density, self.predict_property(test_embeddings, self.data_density)])
            rg = np.concatenate([rg, self.predict_property(test_embeddings, self.data_rg)])
        
        return {'Tg': tg, 'FFV': ffv, 'Tc': tc, 'Density': density, 'Rg': rg}

In [9]:
regressor = NWRegressor(df_train, model, 0.5)
regressor.embeddings = embeddings

In [10]:
# results with kernel bandwidth = 0.5
results = regressor.predict(list(df_test['SMILES']))
results

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'Tg': array([159.85517141, 150.02688145, 153.6889426 ]),
 'FFV': array([0.37095914, 0.37088752, 0.36688773]),
 'Tc': array([0.21484122, 0.22679529, 0.22287344]),
 'Density': array([1.1313129 , 1.12658923, 1.15190977]),
 'Rg': array([20.22171483, 21.099519  , 19.90347482])}

In [11]:
# export into csv
def SaveOutput(results_dict, output_filename, df_test):
    df = pd.DataFrame()
    # get id col from df_test
    df['id'] = df_test['id']
    # get other info from results_dict
    df['Tg'] = results_dict['Tg']
    df['FFV'] = results_dict['FFV']
    df['Tc'] = results_dict['Tc']
    df['Density'] = results_dict['Density']
    df['Rg'] = results_dict['Rg']
    # save to specified location
    df.to_csv(output_filename, index = False)    

In [12]:
SaveOutput(results, 'submission.csv', df_test)